In [1]:
import datasets
from transformers import AutoTokenizer
import tqdm as notebook_tqdm

In [2]:
MAX_LENGTH = 1024
OPTIMAL_NUM_TOKENS = 220_000_000
SEED = 42

In [3]:
raw_datasets = datasets.load_dataset("cerebras/SlimPajama-627B", streaming=True)
tokenizer = AutoTokenizer.from_pretrained("lmsys/vicuna-7b-v1.5")

In [ ]:
shuffled_dataset = raw_datasets.shuffle(seed=SEED)

In [ ]:
tokenized_dataset = {}
for key in raw_datasets.keys():
    tokenized_dataset[key] = shuffled_dataset[key].map(
        lambda x: {
            'tokens': tokenizer(x["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH, return_tensors='pt')
            }
    )

In [ ]:
for data in shuffled_dataset['train']:
    print(data)
    break

In [ ]:
for data in tokenized_dataset['train']:
    print(data)
    break

In [ ]:
list(tokenized_dataset['train'].take(1))